In [1]:
# ce code sert à faire des mesures de dispersion des ondes, en utilisant l'interface python permettant de communiquer avec
# le GBF DG1022, et celui permettant de communiquer avex la camera basler
#%% import libraries

import os
import sys
import time
from time import sleep
import  threading
from multiprocessing import Queue
import platform
from PyQt5.QtGui import *

from pypylon import pylon
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import datetime
#if platform.system() =='Windows':
import pyvisa

In [2]:
#%% functions and classes for interface DG1022
debug_mode = True

def debugPrint(message):
    if debug_mode:
        print(datetime.datetime.utcnow(),' ',message)

DEVICE = '/dev/usbtmc0'

class usbtmc:
    """ Simple implementation of USBTMC device driver """
    def __init__(self,device):
        self.device = device
        try:
            self.FILE = os.open(device, os.O_RDWR)
        except Exception:
            QMessageBox.about(None,'ERROR','Unable to find requried device\n' +device)
            exit()

    def ask(self, message=""):
        """ Send a query to the unit and read the response."""
        self.write(message)
        return self.read()


    def write(self,command):
        """Send an arbitrary command directly to the device"""
        os.write(self.FILE,command)
        sleep(0.2)

    def read(self, length = 4000):
        """ Read an arbitary amount of data directly from the device"""
        return os.read(self.FILE, length)

    def getName(self):
        """ Return the unique identifier based on the *IDN? query"""
        seljf.write("*IDN?")
        sleep(0.10)
        return self.read(300)

    def sendReset(self):
        self.write("*RST")

    def close(self):
        """ Returns the ability for the user to press buttons"""
        self.write("SYST:LOC")


SINE,SQUARE,RAMP,PULSE,NOISE,ARB = range(0,6)
CH1, CH2 = range(6,8)
VPP,VRMS,DBM = range(8,11)

class RigolDG:
    """ Class to control the Rigol DS1000 series oscilloscope"""
    def __init__(self,device):
        self.meas = None
        if platform.system() =='Windows':
            rm = pyvisa.ResourceManager()
            self.meas = rm.open_resource(device)
        else:
            self.meas = usbtmc(device)
        # increase timeout
        self.meas.timeout = 5000

        self.syncVoltage=False
        self.syncFreq=False
        self.syncFreqRatio1_2=1.0
        self.syncVoltageRatio1_2=1.0
        self.reset()


    def getName(self):
        self.write("*IDN?")
        sleep(0.01)
        return self.read()

    def reset(self):
        """ Reset the instrument"""
        self.write("*RST")


    def setFunc(self, function = None, channel = None):
        """ This sets the current function of the generator, SINE, SQUARE, NOISE, RAMP"""
        if channel == None:
            channel = CH1
        if channel == CH1:
            if function == None:
                function = SINE
            msg = "FUNC "
            if(function == SINE):
                msg+="SIN"
            elif(function == SQUARE):
                msg += "SQU"
            elif(function == RAMP):
                msg += "RAMP"
            elif(function == NOISE):
                msg += "NOIS"

            if len(msg) > 6:
                self.write(msg)

        elif channel == CH2:
            if function == None:
                function = SINE
            msg = "FUNC:CH2 "
            if(function == SINE):
                msg+="SIN"
            elif(function == SQUARE):
                msg += "SQU"
            elif(function == RAMP):
                msg += "RAMP"
            elif(function == NOISE):
                msg += "NOIS"

            if len(msg) > 9:
                self.write(msg)
 

    def enableChan1(self, en = True):
        """ This function enables and checks the output of the channel. Ocassionally the screen will time out and the program would have to resend the command to ensure complience. This function only needs to be called once and will retry several times if needed"""
        msg = "OUTPut1 "
        if en:
            msg += "ON"
        else:
            msg += "OFF"
        self.write(msg)
        """attempts = 1
        maxattempts = 5
        

        if en:
            while (self.ask("OUTP?") == "OFF\n\r") and en:
                self.write(msg)
                attempts += 1
                if attempts >= maxattempts:
                    raise ValueException("Unable to Enable Channel 1!")

        else:
            while (self.ask("OUTP?") == "ON\n\r") and not en:
                self.write(msg)
                attempts +=1
                if attempts >= maxattempts:
                    raise ValueException("Unable to Disable Channel 1!")
        """

    def enableChan2(self, en=True):
        """ This function enables and checks the output of the channel. Ocassionally the screen will time out and the program would have to resend the command to ensure complience. This function only needs to be called once and will retry several times if needed"""
        msg = "OUTP:CH2 "
        if en:
            msg += "ON"
        else:
            msg += "OFF"
        self.write(msg)

        """self.Chan2_ON = en
        attempts = 1
        if en:
            while (self.ask("OUTP:CH2?") == "OFF\n\r") and en:
                self.write(msg)
                attempts += 1
                if attempts >= 5:
                    raise ValueException("Unable to Enable Channel 2!")

        else:
            while (self.ask("OUTP:CH2?") == "ON\n\r") and not en:
                self.write(msg)
                attempts +=1
                if attempts >= 5:
                    raise ValueException("Unable to Disable Channel 2!")
        """
    def isChan1Enabled(self):
        """ Check if we've got the ouput enabled"""
        ans = self.ask("OUTP?")
        if ans == 'ON\n\r':
            return True
        else:
            return False

    def isChan2Enabled(self):
        """ Check if we've got the ouput enabled"""
        ans = self.ask("OUTP:CH2?")
        if ans == 'ON\n\r':
            return True
        else:
            return False


    def setFreqHz(self, value=10000.0 ,channel =None):
        """ Set the frequency of the channel. Must provide frequency as a Hz value, ie 0.01Hz or 100000000Hz"""
        if(channel == None):
            if self.syncFreq:
                msg="FREQ "+str(value)
                self.write(msg)
                msg="FREQ:CH2 "+str(value * self.syncFreqRatio1_2)
                self.write(msg)
                return
            else:
                channel = CH1
        if channel == CH1:
            msg = "FREQ "+str(value)

            self.write(msg)
        elif channel == CH2:
            msg = "FREQ:CH2 " + str(value)
            self.write(msg)
        else:
            print("Unsupported channel selection")

    def write(self, message=""):
        """ send the message using the low level control"""
        self.meas.write(message)

    def read(self):
        """ read the message using the low level control"""
        return self.meas.read()

    def ask(self, message=""):
        """ request a response using the low level control"""
        self.write(message)
        sleep(0.1)
        return self.read()

    def setVoltageUnits(self,unit="VPP",channel = None):
        """ Provide string representation of voltage units"""
        if channel == None:
            if self.syncVoltage:
                msg="VOLT:UNIT "+unit
                self.write(msg)
                msg="VOLT:UNIT:CH2 " + unit
                self.write(msg)
                return

            else:
                channel = CH1

        if channel == CH1:
            msg = "VOLT:UNIT " + unit
            self.write(msg)
            msg = "VOLT "+ str(value)
            self.write(msg)
        elif channel == CH2:
            msg = "VOLT:UNIT:CH2 " + unit
            self.write(msg)
            msg = "VOLT:CH2 "+ str(value)
            self.write(msg)

        else:
            print("Unsupported channel selection")


    def setVoltage(self, value = 1.0, channel = None , offset=None):
        """ Set the current voltage and offset for the given channel. If no Channel specified, channel one is used by default"""
        if channel == None:
            if self.syncVoltage:
                msg="VOLT " + str(value)
                self.write(msg)

                msg="VOLT:CH2 "+str((float(value * self.syncVoltageRatio1_2)))
                sleep(0.1)
                self.write(msg)
                if offset is not None:
                    msg = "VOLT:OFFS " + offset
                    self.write(msg)
                if offset is not None:
                    msg = "VOLT:OFFS:CH2 " + str(offset)
                    self.write(msg)
                return

            else:
                channel = CH1

        if channel == CH1:
            msg = "VOLT "+ str(value)
            self.write(msg)
            if offset is not None:
                msg = "VOLT:OFFS " + offset
                self.write(msg)

        elif channel == CH2:
            msg = "VOLT:CH2 "+ str(value)
            self.write(msg)
            if offset is not None:
                msg = "VOLT:OFFS:CH2 " + str(offset)
                self.write(msg)
        else:
            print("Unsupported channel selection")

    def disconnect(self):
        """ Call this when you're done using the unit"""
        self.meas.close()


    def readVoltage(self, channel=None):
        """Read the current value, Channel one used when no channel specified"""
        if channel == None:
            channel = CH1
        volt = 0.0
        if channel == CH1:
            v1 = self.ask("VOLT?")
            end = len(v1)
            volt = float(v1[:end-1])
        elif channel == CH2:
            v2= self.ask("VOLT:CH2?")
            end = len (v2)
            volt = float(v2[4:end-1])
        return volt

    def readFreq(self, channel=None):
        """Read the current value, Channel one used when no channel specified"""
        if channel == None:
            channel = CH1
        #freq = 0.01
        if channel == CH1:
            f1 = self.ask("FREQ?")
            end = len(f1)
            freq = float(f1[:end-1])
        elif channel == CH2:
            f2 = self.ask("FREQ:CH2?")
            end = len(f2)
            freq = float(f2[4:end-1])
        return freq

    def readFunc(self,channel=None):
        """Read the current value, Channel one used when no channel specified"""
        if channel == None:
            channel = CH1
        func =""
        retValue = None
        if channel == CH1:
            func = self.ask("FUNC?")
        elif channel == CH2:
            func = self.ask("FUNC:CH2?")
        if "SIN" in func:
            retValue = SINE
        elif "SQU" in func:
            retValue = SQUARE
        elif "RAMP" in func:
            retValue = RAMP
        elif "NOIS" in func:
            retValue = NOISE
        return retValue

    def syncVoltages(self, sync=True, ratio_CH1 = 1.0, ratio_CH2=1.0):
        """ Handle the sincronisation of the channels here"""
        if sync:
            self.syncVoltage = True
            self.syncVoltageRatio1_2 = ratio_CH2 / ratio_CH1
        else:
            self.syncVoltage = False

    def syncFrequency(self,sync=True, ratio_CH1 = 1.0, ratio_CH2 = 1.0):
        """ Handle the sincronisation of the channels here"""
        if sync:
            self.syncFreq=True
            self.syncFreqRatio1_2 = ratio_CH2 / ratio_CH1
        else:
            self.syncFreq = False

def GetDG1022Device():
    """ Call this function to get cross-platform access for the function generator"""
    if platform.system() == 'Linux':
        return RigolDG('/dev/usbtmc0')
    elif platform.system() == 'Windows':
        rm = pyvisa.ResourceManager()  # This will work with both NI-VISA and pyvisa-py
        usb = list(filter(lambda x: 'USB' in x, rm.list_resources()))
        if len(usb) != 1:
            print('Unable to specify instruments. Please check connection and try again.')
            sys.exit(-1)
        return RigolDG(usb[0])




In [3]:
#%% fonctions de triggering
def configure_external_trigger(camera,idx_cam,isopen=True):
    # Example for setting the TriggerSource to SoftwareSignal1
    if isopen==False:
        camera.Open()

    # Ensure the camera is not grabbing
    if camera.IsGrabbing():
        camera.StopGrabbing()


    camera.TriggerMode.SetValue('On')
    camera.TriggerSource.SetValue(list_trigger_source[idx_cam])
    camera.TriggerDelay.SetValue(list_trigger_delay[idx_cam])
    camera.TriggerSelector.SetValue("FrameStart")

    if isopen==False:
        camera.Close()


In [16]:
# %% interface et acquisitions basler (choix nombre de cameras ici)

# d'abord on choisit nombre de cameras etc.
NUM_CAMERAS = 2

# Create a CameraFactory instance
tl_factory = pylon.TlFactory.GetInstance()

# Get the number of available devices
device_count = tl_factory.EnumerateDevices()
# create the "empty" InstantCameraArray object
cam_array = pylon.InstantCameraArray(NUM_CAMERAS)
print(list(cam_array))

# create as much InstantCamera objects as there are cameras to put in the InstantCameraArray object
#camera1 = pylon.InstantCamera(tl_factory.CreateFirstDevice())
#camera2 = pylon.InstantCamera(tl_factory.CreateDevice(device_count[1]))

# Assign the individual cameras to the array
for i in range(NUM_CAMERAS):
    cam_array[i].Attach(tl_factory.CreateDevice(device_count[i]))
cam_array.Open()
for i in range(NUM_CAMERAS):
    print('camera with index '+str(i) + ' has seral number : '+ cam_array[i].DeviceSerialNumber())
cam_array.Close()


[<pypylon.pylon.InstantCamera; proxy of <Swig Object of type 'Pylon::CInstantCamera *' at 0x000001B45C6C1810> >, <pypylon.pylon.InstantCamera; proxy of <Swig Object of type 'Pylon::CInstantCamera *' at 0x000001B49A5DFED0> >]
camera with index 0 has seral number : 22458101
camera with index 1 has seral number : 23029848


In [17]:
#%% ensuite assigner à chaque caméra une liste pour chaque parametre à changer (les listes doivent chacune avoir NUM_CAMERAS elements)
# choose camera settings
#freq_acq = 10  # Replace with your desired frame rate (Hz)
list_exposure_time = [2000,2000]  # Replace with your desired exposure time in microseconds
list_width =  [1280,1280] # Replace with your desired width (px) (doit etre multiple de 16)
list_height = [1024,1024]   # Replace with your desired height (px) (peut etre de nimporte quelle taille)
list_offset_x = [8,8]  # Replace with your desired X offset
list_offset_y = [8,8]  # Replace with your desired Y offset
list_trigger_source = ['Line4','Line4'] 
list_trigger_delay = [0,0]  

#camera.MaxNumBuffer.SetValue(desired_buffer_size) (fonction pour choisir buffer size)

# %% choose number of images to grab and the name of the daily folder
numberOfImagesToGrab = 9000 # choisir nombre d'images
save_images = True
change_dimensions = False

#dailyfolder = 'D:/Vasco/Frigo_pmmh/test_interface_camera/test_two_basler/'
#dailyfolder = 'D:/Vasco/Frigo_pmmh/novembre2024/20241105/manip_f_k_membrane/'
#dailyfolder = 'D:/manip_grenoble2024/manips_relation_dispersion/20241125/'
dailyfolder = 'D:/Grenoble/20241128/manip_fracture/'
num_acquisition = 6

acquisition_folder = dailyfolder + 'Acquisition_' + str(num_acquisition) + '/'


#if os.path.exists(savefolder+'images/')==False:
#    os.mkdir(savefolder+'images/')

# %% def the function make_acquisition (for a given frequency, for all chosen cameras)
def make_acquisition(f_exc,freq_acq):
    global cam_array
    global GBF
    try:
        cam_array.Open()

        # demonstrate some feature access
        list_str_camera_serial_number = []
        #list_freq_acq_effective = []
        for idx_cam in range(len(list(cam_array))):
            camera = cam_array[idx_cam]
            print(camera.Width.GetInc())
            print(camera.Width.GetValue())
            #camera.AcquisitionFrameRate.SetValue(freq_acq)  # Replace with your desired frame rate
            #camera.AcquisitionFrameRateEnable.SetValue(True)            
            #print('frame rate : ',camera.ResultingFrameRate.GetValue())
            camera.ExposureTime.SetValue(list_exposure_time[idx_cam])  # Replace with your desired exposure time in microseconds

            if change_dimensions==True:    
                camera.Width.SetValue(list_width[idx_cam])  # Replace with your desired width
                camera.Height.SetValue(list_height[idx_cam])  # Replace with your desired height
                camera.OffsetX.SetValue(list_offset_x[idx_cam])  # Replace with your desired X offset
                camera.OffsetY.SetValue(list_offset_y[idx_cam])  # Replace with your desired Y offset
            elif change_dimensions==False:
                new_width = camera.Width.GetValue() - camera.Width.GetInc()
                if new_width >= camera.Width.GetMin():
                    camera.Width.SetValue(new_width)
            # activer le trigger externe sur camera :
            configure_external_trigger(camera,idx_cam)
            # mettre un buffer assez grand :
            camera.MaxNumBuffer.SetValue(1000)
            str_camera_serial_number = camera.DeviceSerialNumber()
            list_str_camera_serial_number.append(str_camera_serial_number)
            #list_freq_acq_effective.append(freq_acq_effective)
            savefolder = acquisition_folder+'camera_'+str_camera_serial_number+'/' + str(f_exc)+'Hz_'+ str(np.round(freq_acq,4))+'Hz/'

            if save_images:
                if os.path.exists(acquisition_folder)==False:
                    os.mkdir(acquisition_folder)
                if os.path.exists(acquisition_folder+'camera_'+str_camera_serial_number+'/')==False:
                    os.mkdir(acquisition_folder+'camera_'+str_camera_serial_number+'/')
                if os.path.exists(savefolder)==False:
                    os.mkdir(savefolder)
                if os.path.exists(savefolder + 'images/')==False:
                    os.mkdir(savefolder + 'images/')

        # send a digital pulse to only the first camera, and all other cameras should be synchonized to it
        #send_pulse_to_camera(cam_array[0])
        # note the date and time of the beginning of the acquisition (useful in file names)
        time_acquisition = datetime.datetime.now()
        year = time_acquisition.year
        month = time_acquisition.month
        day = time_acquisition.day
        hour = time_acquisition.hour
        minutes = time_acquisition.minute
        seconds = time_acquisition.second
        str_time_acquisition = str(year)+str(month).zfill(2)+str(day).zfill(2)+'_'+str(hour).zfill(2)+str(minutes).zfill(2)+str(seconds).zfill(2)
        
        

        # start to grab images for all cameras at the same time
        #cam_array.StartGrabbing(numberOfImagesToGrab)
        array_time_elapsed = []
        time_init = time.time()
        for camera in cam_array:
            camera.StartGrabbing()
            array_time_elapsed.append(time.time() - time_init)
        array_time_elapsed = np.array(array_time_elapsed)
        print(array_time_elapsed)
        array_time_elapsed_wrtcam0 = array_time_elapsed - array_time_elapsed[0]
        array_num_images_elapsed = array_time_elapsed_wrtcam0//(1/freq_acq)
        print(array_time_elapsed_wrtcam0)
        print(array_num_images_elapsed)
            
        i = 0

        while cam_array[0].IsGrabbing(): # on a dû adapter :/ (normalement c'était juste cam_array.IsGrabbing)
            for idx_cam in range(len(list(cam_array))):
                camera = cam_array[idx_cam]
                grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
                if grabResult.GrabSucceeded():
                    # Access the image data.
                    print('idx_cam =',idx_cam)
                    #print("SizeX: ", grabResult.Width)
                    #print("SizeY: ", grabResult.Height)
                    img = grabResult.Array
                    #print("Gray value of first pixel: ", img[0, 0])
                    if save_images==True:
                        # Convert the NumPy array to a PIL Image object
                        image = Image.fromarray(img)
                        # Save the image as a TIFF file (black and white)
                        str_camera_serial_number = list_str_camera_serial_number[idx_cam]
                        savefolder = acquisition_folder+'camera_'+str_camera_serial_number+'/' + str(f_exc)+'Hz_' + str(np.round(freq_acq,4))+'Hz/'

                        namefile = 'Basler__'+str_camera_serial_number+'__'+str_time_acquisition+'_'+str(int(i+array_num_images_elapsed[idx_cam]-array_num_images_elapsed[-1])).zfill(4)+'.tiff' # pour les cameras qui commencent avant les autres, les premieres images auront des nombres négatifs
                        #namefile = 'Basler__'+str_camera_serial_number+'__'+str_time_acquisition+'_'+str(int(i)).zfill(4)+'.tiff'
                        if (int(i+array_num_images_elapsed[idx_cam]-array_num_images_elapsed[-1]) < 0)|(int(i+array_num_images_elapsed[idx_cam]-array_num_images_elapsed[-1])>=numberOfImagesToGrab):
                            pass
                        else:
                            image.save(savefolder+'images/'+namefile)
                        
            grabResult.Release()
            print(i)
            i+=1
            if i>=numberOfImagesToGrab + array_num_images_elapsed[-1]:
                cam_array.StopGrabbing()
                cam_array.Close()            

    finally:
        # Ensure camera is closed properly
        #cam_array.StopGrabbing()
        cam_array.Close()



In [18]:
# %% "initialise la manip" : permet de tester amplitude pot vibrant avant de lancer toutes les frequences notamment

# mettre une condition sur si on veut faire de la strobo ou pas...
# fonction qui calcule la frequence d'acquisition permettant d'avoir nbpoints=100 points par periode en strobo
def calcul_freq_ech(f,npoints=100,fthr=161):
    T = 1/f
    Tvideo = T * ((npoints+1)/npoints + int(f/fthr))
    fvideo = 1/Tvideo
    return (npoints * Tvideo,fvideo)


List_f_exc = [1.34]
#List_f_exc = np.arange(1,11,1)
List_freq_acq = [15]
for f_exc in List_f_exc:
    List_freq_acq.append(calcul_freq_ech(f_exc,npoints=100,fthr=161)[1])

#List_freq_acq = [157.1]
#List_freq_acq = np.ones(10)*57.7

# cette partie peut etre lancee avant juste pour tester la communication avec le GBF

GBF = GetDG1022Device()
#sleep(1)
#GBF.setVoltage(value=0.1,channel=CH1) # voir en fonctiion de la frequence quelle tension appliquer !!!
#GBF.setVoltage(value=0.1,channel=CH1) # voir en fonctiion de la frequence quelle tension appliquer !!!
sleep(1)
GBF.setVoltage(value=5,channel=CH2)
sleep(1)
GBF.setFunc(function=SQUARE,channel=CH2)
#sleep(1)
#GBF.setFunc(function=SINE,channel=CH1)
sleep(1)
GBF.setFreqHz(value=List_f_exc[0],channel=CH1)
sleep(1)
GBF.setFreqHz(value=np.round(List_freq_acq[0],4),channel=CH2)
sleep(1)
GBF.enableChan2(en=True)
sleep(1)
#GBF.enableChan1(en=True)


In [19]:

#%% boucle qui change les freq
for i in range(len(List_f_exc)):
#        global camera
    # allumer channel 1 !!!!!!!!!!
    # imposer sur channel 2 la frequence freq_acq
    #if GBF.isChan2Enabled()==False:
    #    GBF.enableChan2(en=True)
    #if GBF.isChan1Enabled==False:

    #    GBF.enableChan1(en=True)
    sleep(1)
    GBF.setFreqHz(value=np.round(List_freq_acq[i],4),channel=CH2) # set freq acq ch2
    sleep(1)
    GBF.enableChan2(en=True) # activate ch2
    #sleep(1)
    #GBF.setFreqHz(value=List_f_exc[i],channel=CH1) # set f_exc ch1
    #sleep(1)
    #GBF.enableChan1(en=True) # activate ch1
    sleep(1)
    make_acquisition(List_f_exc[i],List_freq_acq[i]) # lancer acquisition pour un couple (f_exc,freq_acq) donné
    #sleep(1)
    #GBF.enableChan1(en=False) # éteindre ch1
    #sleep(1)
    #GBF.enableChan2(en=False) # éteindre ch2
    #sleep(1)


8
2048
8
2048
[0.03914547 0.15137649]
[0.         0.11223102]
[0. 1.]
idx_cam = 0
idx_cam = 1
0
idx_cam = 0
idx_cam = 1
1
idx_cam = 0
idx_cam = 1
2
idx_cam = 0
idx_cam = 1
3
idx_cam = 0
idx_cam = 1
4
idx_cam = 0
idx_cam = 1
5
idx_cam = 0
idx_cam = 1
6
idx_cam = 0
idx_cam = 1
7
idx_cam = 0
idx_cam = 1
8
idx_cam = 0
idx_cam = 1
9
idx_cam = 0
idx_cam = 1
10
idx_cam = 0
idx_cam = 1
11
idx_cam = 0
idx_cam = 1
12
idx_cam = 0
idx_cam = 1
13
idx_cam = 0
idx_cam = 1
14
idx_cam = 0
idx_cam = 1
15
idx_cam = 0
idx_cam = 1
16
idx_cam = 0
idx_cam = 1
17
idx_cam = 0
idx_cam = 1
18
idx_cam = 0
idx_cam = 1
19
idx_cam = 0
idx_cam = 1
20
idx_cam = 0
idx_cam = 1
21
idx_cam = 0
idx_cam = 1
22
idx_cam = 0
idx_cam = 1
23
idx_cam = 0
idx_cam = 1
24
idx_cam = 0
idx_cam = 1
25
idx_cam = 0
idx_cam = 1
26
idx_cam = 0
idx_cam = 1
27
idx_cam = 0
idx_cam = 1
28
idx_cam = 0
idx_cam = 1
29
idx_cam = 0
idx_cam = 1
30
idx_cam = 0
idx_cam = 1
31
idx_cam = 0
idx_cam = 1
32
idx_cam = 0
idx_cam = 1
33
idx_cam = 0
idx_cam = 

idx_cam = 1
294
idx_cam = 0
idx_cam = 1
295
idx_cam = 0
idx_cam = 1
296
idx_cam = 0
idx_cam = 1
297
idx_cam = 0
idx_cam = 1
298
idx_cam = 0
idx_cam = 1
299
idx_cam = 0
idx_cam = 1
300
idx_cam = 0
idx_cam = 1
301
idx_cam = 0
idx_cam = 1
302
idx_cam = 0
idx_cam = 1
303
idx_cam = 0
idx_cam = 1
304
idx_cam = 0
idx_cam = 1
305
idx_cam = 0
idx_cam = 1
306
idx_cam = 0
idx_cam = 1
307
idx_cam = 0
idx_cam = 1
308
idx_cam = 0
idx_cam = 1
309
idx_cam = 0
idx_cam = 1
310
idx_cam = 0
idx_cam = 1
311
idx_cam = 0
idx_cam = 1
312
idx_cam = 0
idx_cam = 1
313
idx_cam = 0
idx_cam = 1
314
idx_cam = 0
idx_cam = 1
315
idx_cam = 0
idx_cam = 1
316
idx_cam = 0
idx_cam = 1
317
idx_cam = 0
idx_cam = 1
318
idx_cam = 0
idx_cam = 1
319
idx_cam = 0
idx_cam = 1
320
idx_cam = 0
idx_cam = 1
321
idx_cam = 0
idx_cam = 1
322
idx_cam = 0
idx_cam = 1
323
idx_cam = 0
idx_cam = 1
324
idx_cam = 0
idx_cam = 1
325
idx_cam = 0
idx_cam = 1
326
idx_cam = 0
idx_cam = 1
327
idx_cam = 0
idx_cam = 1
328
idx_cam = 0
idx_cam = 1
329
idx_

587
idx_cam = 0
idx_cam = 1
588
idx_cam = 0
idx_cam = 1
589
idx_cam = 0
idx_cam = 1
590
idx_cam = 0
idx_cam = 1
591
idx_cam = 0
idx_cam = 1
592
idx_cam = 0
idx_cam = 1
593
idx_cam = 0
idx_cam = 1
594
idx_cam = 0
idx_cam = 1
595
idx_cam = 0
idx_cam = 1
596
idx_cam = 0
idx_cam = 1
597
idx_cam = 0
idx_cam = 1
598
idx_cam = 0
idx_cam = 1
599
idx_cam = 0
idx_cam = 1
600
idx_cam = 0
idx_cam = 1
601
idx_cam = 0
idx_cam = 1
602
idx_cam = 0
idx_cam = 1
603
idx_cam = 0
idx_cam = 1
604
idx_cam = 0
idx_cam = 1
605
idx_cam = 0
idx_cam = 1
606
idx_cam = 0
idx_cam = 1
607
idx_cam = 0
idx_cam = 1
608
idx_cam = 0
idx_cam = 1
609
idx_cam = 0
idx_cam = 1
610
idx_cam = 0
idx_cam = 1
611
idx_cam = 0
idx_cam = 1
612
idx_cam = 0
idx_cam = 1
613
idx_cam = 0
idx_cam = 1
614
idx_cam = 0
idx_cam = 1
615
idx_cam = 0
idx_cam = 1
616
idx_cam = 0
idx_cam = 1
617
idx_cam = 0
idx_cam = 1
618
idx_cam = 0
idx_cam = 1
619
idx_cam = 0
idx_cam = 1
620
idx_cam = 0
idx_cam = 1
621
idx_cam = 0
idx_cam = 1
622
idx_cam = 0
idx_

881
idx_cam = 0
idx_cam = 1
882
idx_cam = 0
idx_cam = 1
883
idx_cam = 0
idx_cam = 1
884
idx_cam = 0
idx_cam = 1
885
idx_cam = 0
idx_cam = 1
886
idx_cam = 0
idx_cam = 1
887
idx_cam = 0
idx_cam = 1
888
idx_cam = 0
idx_cam = 1
889
idx_cam = 0
idx_cam = 1
890
idx_cam = 0
idx_cam = 1
891
idx_cam = 0
idx_cam = 1
892
idx_cam = 0
idx_cam = 1
893
idx_cam = 0
idx_cam = 1
894
idx_cam = 0
idx_cam = 1
895
idx_cam = 0
idx_cam = 1
896
idx_cam = 0
idx_cam = 1
897
idx_cam = 0
idx_cam = 1
898
idx_cam = 0
idx_cam = 1
899
idx_cam = 0
idx_cam = 1
900
idx_cam = 0
idx_cam = 1
901
idx_cam = 0
idx_cam = 1
902
idx_cam = 0
idx_cam = 1
903
idx_cam = 0
idx_cam = 1
904
idx_cam = 0
idx_cam = 1
905
idx_cam = 0
idx_cam = 1
906
idx_cam = 0
idx_cam = 1
907
idx_cam = 0
idx_cam = 1
908
idx_cam = 0
idx_cam = 1
909
idx_cam = 0
idx_cam = 1
910
idx_cam = 0
idx_cam = 1
911
idx_cam = 0
idx_cam = 1
912
idx_cam = 0
idx_cam = 1
913
idx_cam = 0
idx_cam = 1
914
idx_cam = 0
idx_cam = 1
915
idx_cam = 0
idx_cam = 1
916
idx_cam = 0
idx_

1170
idx_cam = 0
idx_cam = 1
1171
idx_cam = 0
idx_cam = 1
1172
idx_cam = 0
idx_cam = 1
1173
idx_cam = 0
idx_cam = 1
1174
idx_cam = 0
idx_cam = 1
1175
idx_cam = 0
idx_cam = 1
1176
idx_cam = 0
idx_cam = 1
1177
idx_cam = 0
idx_cam = 1
1178
idx_cam = 0
idx_cam = 1
1179
idx_cam = 0
idx_cam = 1
1180
idx_cam = 0
idx_cam = 1
1181
idx_cam = 0
idx_cam = 1
1182
idx_cam = 0
idx_cam = 1
1183
idx_cam = 0
idx_cam = 1
1184
idx_cam = 0
idx_cam = 1
1185
idx_cam = 0
idx_cam = 1
1186
idx_cam = 0
idx_cam = 1
1187
idx_cam = 0
idx_cam = 1
1188
idx_cam = 0
idx_cam = 1
1189
idx_cam = 0
idx_cam = 1
1190
idx_cam = 0
idx_cam = 1
1191
idx_cam = 0
idx_cam = 1
1192
idx_cam = 0
idx_cam = 1
1193
idx_cam = 0
idx_cam = 1
1194
idx_cam = 0
idx_cam = 1
1195
idx_cam = 0
idx_cam = 1
1196
idx_cam = 0
idx_cam = 1
1197
idx_cam = 0
idx_cam = 1
1198
idx_cam = 0
idx_cam = 1
1199
idx_cam = 0
idx_cam = 1
1200
idx_cam = 0
idx_cam = 1
1201
idx_cam = 0
idx_cam = 1
1202
idx_cam = 0
idx_cam = 1
1203
idx_cam = 0
idx_cam = 1
1204
idx_cam =

...

7393
idx_cam = 0
idx_cam = 1
7394
idx_cam = 0
idx_cam = 1
7395
idx_cam = 0
idx_cam = 1
7396
idx_cam = 0
idx_cam = 1
7397
idx_cam = 0
idx_cam = 1
7398
idx_cam = 0
idx_cam = 1
7399
idx_cam = 0
idx_cam = 1
7400
idx_cam = 0
idx_cam = 1
7401
idx_cam = 0
idx_cam = 1
7402
idx_cam = 0
idx_cam = 1
7403
idx_cam = 0
idx_cam = 1
7404
idx_cam = 0
idx_cam = 1
7405
idx_cam = 0
idx_cam = 1
7406
idx_cam = 0
idx_cam = 1
7407
idx_cam = 0
idx_cam = 1
7408
idx_cam = 0
idx_cam = 1
7409
idx_cam = 0
idx_cam = 1
7410
idx_cam = 0
idx_cam = 1
7411
idx_cam = 0
idx_cam = 1
7412
idx_cam = 0
idx_cam = 1
7413
idx_cam = 0
idx_cam = 1
7414
idx_cam = 0
idx_cam = 1
7415
idx_cam = 0
idx_cam = 1
7416
idx_cam = 0
idx_cam = 1
7417
idx_cam = 0
idx_cam = 1
7418
idx_cam = 0
idx_cam = 1
7419
idx_cam = 0
idx_cam = 1
7420
idx_cam = 0
idx_cam = 1
7421
idx_cam = 0
idx_cam = 1
7422
idx_cam = 0
idx_cam = 1
7423
idx_cam = 0
idx_cam = 1
7424
idx_cam = 0
idx_cam = 1
7425
idx_cam = 0
idx_cam = 1
7426
idx_cam = 0
idx_cam = 1
7427
idx_cam =

idx_cam = 1
7677
idx_cam = 0
idx_cam = 1
7678
idx_cam = 0
idx_cam = 1
7679
idx_cam = 0
idx_cam = 1
7680
idx_cam = 0
idx_cam = 1
7681
idx_cam = 0
idx_cam = 1
7682
idx_cam = 0
idx_cam = 1
7683
idx_cam = 0
idx_cam = 1
7684
idx_cam = 0
idx_cam = 1
7685
idx_cam = 0
idx_cam = 1
7686
idx_cam = 0
idx_cam = 1
7687
idx_cam = 0
idx_cam = 1
7688
idx_cam = 0
idx_cam = 1
7689
idx_cam = 0
idx_cam = 1
7690
idx_cam = 0
idx_cam = 1
7691
idx_cam = 0
idx_cam = 1
7692
idx_cam = 0
idx_cam = 1
7693
idx_cam = 0
idx_cam = 1
7694
idx_cam = 0
idx_cam = 1
7695
idx_cam = 0
idx_cam = 1
7696
idx_cam = 0
idx_cam = 1
7697
idx_cam = 0
idx_cam = 1
7698
idx_cam = 0
idx_cam = 1
7699
idx_cam = 0
idx_cam = 1
7700
idx_cam = 0
idx_cam = 1
7701
idx_cam = 0
idx_cam = 1
7702
idx_cam = 0
idx_cam = 1
7703
idx_cam = 0
idx_cam = 1
7704
idx_cam = 0
idx_cam = 1
7705
idx_cam = 0
idx_cam = 1
7706
idx_cam = 0
idx_cam = 1
7707
idx_cam = 0
idx_cam = 1
7708
idx_cam = 0
idx_cam = 1
7709
idx_cam = 0
idx_cam = 1
7710
idx_cam = 0
idx_cam = 1
77

idx_cam = 1
7961
idx_cam = 0
idx_cam = 1
7962
idx_cam = 0
idx_cam = 1
7963
idx_cam = 0
idx_cam = 1
7964
idx_cam = 0
idx_cam = 1
7965
idx_cam = 0
idx_cam = 1
7966
idx_cam = 0
idx_cam = 1
7967
idx_cam = 0
idx_cam = 1
7968
idx_cam = 0
idx_cam = 1
7969
idx_cam = 0
idx_cam = 1
7970
idx_cam = 0
idx_cam = 1
7971
idx_cam = 0
idx_cam = 1
7972
idx_cam = 0
idx_cam = 1
7973
idx_cam = 0
idx_cam = 1
7974
idx_cam = 0
idx_cam = 1
7975
idx_cam = 0
idx_cam = 1
7976
idx_cam = 0
idx_cam = 1
7977
idx_cam = 0
idx_cam = 1
7978
idx_cam = 0
idx_cam = 1
7979
idx_cam = 0
idx_cam = 1
7980
idx_cam = 0
idx_cam = 1
7981
idx_cam = 0
idx_cam = 1
7982
idx_cam = 0
idx_cam = 1
7983
idx_cam = 0
idx_cam = 1
7984
idx_cam = 0
idx_cam = 1
7985
idx_cam = 0
idx_cam = 1
7986
idx_cam = 0
idx_cam = 1
7987
idx_cam = 0
idx_cam = 1
7988
idx_cam = 0
idx_cam = 1
7989
idx_cam = 0
idx_cam = 1
7990
idx_cam = 0
idx_cam = 1
7991
idx_cam = 0
idx_cam = 1
7992
idx_cam = 0
idx_cam = 1
7993
idx_cam = 0
idx_cam = 1
7994
idx_cam = 0
idx_cam = 1
79

idx_cam = 1
8245
idx_cam = 0
idx_cam = 1
8246
idx_cam = 0
idx_cam = 1
8247
idx_cam = 0
idx_cam = 1
8248
idx_cam = 0
idx_cam = 1
8249
idx_cam = 0
idx_cam = 1
8250
idx_cam = 0
idx_cam = 1
8251
idx_cam = 0
idx_cam = 1
8252
idx_cam = 0
idx_cam = 1
8253
idx_cam = 0
idx_cam = 1
8254
idx_cam = 0
idx_cam = 1
8255
idx_cam = 0
idx_cam = 1
8256
idx_cam = 0
idx_cam = 1
8257
idx_cam = 0
idx_cam = 1
8258
idx_cam = 0
idx_cam = 1
8259
idx_cam = 0
idx_cam = 1
8260
idx_cam = 0
idx_cam = 1
8261
idx_cam = 0
idx_cam = 1
8262
idx_cam = 0
idx_cam = 1
8263
idx_cam = 0
idx_cam = 1
8264
idx_cam = 0
idx_cam = 1
8265
idx_cam = 0
idx_cam = 1
8266
idx_cam = 0
idx_cam = 1
8267
idx_cam = 0
idx_cam = 1
8268
idx_cam = 0
idx_cam = 1
8269
idx_cam = 0
idx_cam = 1
8270
idx_cam = 0
idx_cam = 1
8271
idx_cam = 0
idx_cam = 1
8272
idx_cam = 0
idx_cam = 1
8273
idx_cam = 0
idx_cam = 1
8274
idx_cam = 0
idx_cam = 1
8275
idx_cam = 0
idx_cam = 1
8276
idx_cam = 0
idx_cam = 1
8277
idx_cam = 0
idx_cam = 1
8278
idx_cam = 0
idx_cam = 1
82

8528
idx_cam = 0
idx_cam = 1
8529
idx_cam = 0
idx_cam = 1
8530
idx_cam = 0
idx_cam = 1
8531
idx_cam = 0
idx_cam = 1
8532
idx_cam = 0
idx_cam = 1
8533
idx_cam = 0
idx_cam = 1
8534
idx_cam = 0
idx_cam = 1
8535
idx_cam = 0
idx_cam = 1
8536
idx_cam = 0
idx_cam = 1
8537
idx_cam = 0
idx_cam = 1
8538
idx_cam = 0
idx_cam = 1
8539
idx_cam = 0
idx_cam = 1
8540
idx_cam = 0
idx_cam = 1
8541
idx_cam = 0
idx_cam = 1
8542
idx_cam = 0
idx_cam = 1
8543
idx_cam = 0
idx_cam = 1
8544
idx_cam = 0
idx_cam = 1
8545
idx_cam = 0
idx_cam = 1
8546
idx_cam = 0
idx_cam = 1
8547
idx_cam = 0
idx_cam = 1
8548
idx_cam = 0
idx_cam = 1
8549
idx_cam = 0
idx_cam = 1
8550
idx_cam = 0
idx_cam = 1
8551
idx_cam = 0
idx_cam = 1
8552
idx_cam = 0
idx_cam = 1
8553
idx_cam = 0
idx_cam = 1
8554
idx_cam = 0
idx_cam = 1
8555
idx_cam = 0
idx_cam = 1
8556
idx_cam = 0
idx_cam = 1
8557
idx_cam = 0
idx_cam = 1
8558
idx_cam = 0
idx_cam = 1
8559
idx_cam = 0
idx_cam = 1
8560
idx_cam = 0
idx_cam = 1
8561
idx_cam = 0
idx_cam = 1
8562
idx_cam =

In [12]:
    
#%% eteindre les deux channels
sleep(1)
GBF.enableChan1(en=False)
sleep(1)
#GBF.enableChan2(en=False)

In [24]:
np.flip(np.arange(10))

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])